In [1]:
import os
from dotenv import load_dotenv
import openai
from dataclasses import dataclass, field
from typing import List
import json
from system_prompts import SYSTEM_PROMPT_REPORT_STRUCTURE, SYSTEM_PROMPT_FIRST_SEARCH, SYSTEM_PROMPT_FIRST_SUMMARY, SYSTEM_PROMPT_REFLECTION
from search_tool import tavily_search

### 加载环境变量和预定义函数及数据结构

In [ ]:
load_dotenv()

@dataclass
class Search:
    url: str = ""
    content: str = ""
    
@dataclass
class Research:
    search_history: List[Search] = field(default_factory=list)
    latest_summary: str = ""
    reflection_iteration: int = 0

@dataclass
class Paragraph:
    title: str = ""
    content: str = ""
    research: Research = field(default_factory=Research)

@dataclass
class State:
    report_title: str = ""
    paragraphs: List[Paragraph] = field(default_factory=list)


def clean_json_tags(json_str):
    return json_str.replace("```json", "").replace("```", "").strip()

def update_state_with_search_results(search_results, idx_paragraph, state):

    for search_result in search_results["results"]:
        search = Search(url=search_result["url"], content=search_result["content"])
        state.paragraphs[idx_paragraph].research.search_history.append(search)
    return state

client = openai.OpenAI(
    api_key=os.getenv("DEEPSEEK_API_KEY"),
    base_url="https://api.deepseek.com"
    )

### 报告大纲智能体

In [19]:
STATE = State()

research_input = "请帮我调研一下web3.0的开发"

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT_REPORT_STRUCTURE},
        {"role": "user", "content": research_input}
    ]
)

report_structure = json.loads(clean_json_tags(response.choices[0].message.content))
for paragraph in report_structure:
    STATE.paragraphs.append(Paragraph(title=paragraph["title"], content=paragraph["content"]))

print("报告大纲：")
for i, paragraph in enumerate(STATE.paragraphs):
    print(f"第{i}个段落标题: {paragraph.title}") 
    print(f"第{i}个段落内容: {paragraph.content}")

报告大纲：
第0个段落标题: Web3.0的定义与核心概念
第0个段落内容: Web3.0是基于区块链技术的下一代互联网，强调去中心化、用户数据主权和智能合约。
第1个段落标题: Web3.0的主要技术栈
第1个段落内容: Web3.0开发涉及的主要技术包括区块链、智能合约、去中心化存储（如IPFS）和去中心化身份（DID）。
第2个段落标题: Web3.0开发工具与框架
第2个段落内容: 常用的Web3.0开发工具和框架包括以太坊开发工具（如Truffle、Hardhat）、Solidity编程语言和Web3.js库。
第3个段落标题: Web3.0的应用场景
第3个段落内容: Web3.0的应用场景包括去中心化金融（DeFi）、非同质化代币（NFT）、去中心化自治组织（DAO）等。
第4个段落标题: Web3.0开发的挑战与解决方案
第4个段落内容: Web3.0开发面临的挑战包括可扩展性、安全性和用户体验问题，以及相应的解决方案。
第5个段落标题: Web3.0开发的未来趋势
第5个段落内容: Web3.0开发的未来趋势可能包括跨链技术、Layer 2解决方案和更广泛的企业应用。
第6个段落标题: 学习Web3.0开发的资源与社区
第6个段落内容: 推荐的学习资源和社区包括官方文档、在线课程（如CryptoZombies）、开发者论坛（如以太坊社区）和黑客松活动。


### 搜索智能体

In [21]:
input_json_first_search = {
    "title": STATE.paragraphs[0].title,
    "content": STATE.paragraphs[0].content
}


response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=[
        {"role":"system","content":SYSTEM_PROMPT_FIRST_SEARCH},
        {"role":"user","content":json.dumps(input_json_first_search)}
        ],
    temperature=1
    )

search_interface = response.choices[0].message.content
search_input = json.loads(search_interface)

print(search_input["search_query"])

Web3.0 核心概念详解：去中心化、用户数据主权、智能合约


### 搜索工具

In [24]:
search_keyword = search_input["search_query"]

search_output = tavily_search(search_input["search_query"])

STATE = update_state_with_search_results(search_output, 0, STATE)


for search in STATE.paragraphs[0].research.search_history:
    print(search.url)
    print(search.content)
    break


https://qianfanmarket.baidu.com/article/detail/1069116
去中心化：借助区块链技术，Web3实现了数据的去中心化存储和传输，有效降低了单点故障的风险，并提高了系统的抗攻击能力。 · 用户数据主权：在Web3时代，用户数据不再是各大平台的


### 段落输出智能体

In [ ]:
input_json_search_results = {
    "title": STATE.paragraphs[0].title,
    "content": STATE.paragraphs[0].content,
    "search_query": search_input["search_query"],
    "search_results": [ result["raw_content"] for result in search_output["results"] if result["raw_content"]]
}


response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=[
        {"role":"system","content":SYSTEM_PROMPT_FIRST_SUMMARY},
        {"role":"user","content":json.dumps(input_json_first_search)}
        ],
    temperature=1
    )

paragraph_output = json.loads(response.choices[0].message.content)

print(paragraph_output["paragraph_latest_state"])

### 反思智能体

In [ ]:
input_json_reflection = {
    "title": STATE.paragraphs[0].title,
    "content": STATE.paragraphs[0].content,
    "paragraph_latest_state": paragraph_output["paragraph_latest_state"]
}

response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=[
        {"role":"system","content":SYSTEM_PROMPT_REFLECTION},
        {"role":"user","content":json.dumps(input_json_reflection)}
        ],
    temperature=1
    )
    
reflection_output = json.loads(response.choices[0].message.content)

print(reflection_output["search_query"])
